# Julia для Data Science

* Данные
* **Обработка данных**
* Визуализация

### Обработка данных: стандартные алгоритмы машинного обучения в Julia

Далее мы увидим, как использовать некоторые стандартные алгоритмы машинного обучения, реализованные в Julia.

In [ ]:
using DataFrames

### Пример 1: кластеризация Kmeans

Давайте начнем с загрузки каких-нибудь данных.

Файл с транзакциями с недвижимостью в Сакраменто, который мы загружаем ниже, представляет собой список из 985 транзакций с недвижимостью в районе Сакраменто, о которых было сообщено в течение пяти дней,

In [ ]:
#download("http://samplecsvs.s3.amazonaws.com/Sacramentorealestatetransactions.csv","houses.csv")
houses = readtable("houses.csv")

Для графиков подключим Plots

In [ ]:
using Plots
plot(size=(500,500),leg=false)

Теперь давайте создадим точечный график, чтобы показать цену дома в зависимости от площади ~~Ленина~~,

In [ ]:
x = houses[:sq__ft]
y = houses[:price]
scatter(x,y,markersize=3)

*Какова цена дома площадью 0 квадратных футов?*

Наверно, те кто составлял таблицу для домов, у которых площадь по бумагам в квадратных метрах, ставили в колонку sq__ft нули.

Но отфильтровать эти дома легко!

In [ ]:
filter_houses = houses[houses[:sq__ft].>0,:]
x = filter_houses[:sq__ft]
y = filter_houses[:price]
scatter(x,y)

Теперь выглядит осмысленней! Чем выше квадратный метраж, тем выше цена.

Мы также можем фильтровать DataFrame по значению функции, используя функцию by.

In [ ]:
by(filter_houses,:type,size)

In [ ]:
using Statistics

In [ ]:
# посмотрим средние цены для типов домов
by(filter_houses,:type,filter_houses->mean(filter_houses[:price]))

Теперь давайте сделаем несколько кластеров kmeans на этих данных.

Во-первых, мы можем загрузить пакет `Clustering` для этого.

In [ ]:
]add Clustering

In [ ]:
using Clustering

Давайте сохраним функции `:широта` и`:долгота` в массиве `X`, который мы передадим в` kmeans`.

Сначала мы добавляем данные для `:latitude` и `:longitude` в новый `DataFrame` с именем` X`.

In [ ]:
X = filter_houses[[:latitude,:longitude]]

а затем мы конвертируем `X` в` Array` используя

```julia
X = Array(X)
```
или
```julia
X = convert(Array, X)
```

Особенно,

```julia
X = Array{Float64}(X)#не работает
```
или же
```julia
X = convert(Array{Float64}, X)#варнинг
```
превратит `X` в` Array`, который хранит объекты типа `Float64`.

In [ ]:
X = convert(Matrix{Float64}, X) # исправил

Каждая функция хранится в виде строки `X`, но мы можем транспонировать, чтобы сделать эти функции столбцами` X`.

In [ ]:
X = X'
X = convert(Matrix{Float64}, X) # исправил

В качестве первого шага, чтобы угадать, сколько кластеров нам может понадобиться, давайте используем количество почтовых индексов в наших данных.

(Попробуйте поварьировать, чтобы увидеть, как это влияет на результаты!)

In [ ]:
k = length(unique(filter_houses[:zip])) 

Мы можем использовать функцию `kmeans` для кластеризации kmeans!

In [ ]:
C = kmeans(X,k) # попробуйте поменять к

Теперь давайте создадим новый фрейм данных, `df`, со всеми теми же данными, что и` filter_houses`, который также включает столбец для кластера, которому был назначен каждый дом.

In [ ]:
df = DataFrame(cluster = C.assignments,city = filter_houses[:city],
    latitude = filter_houses[:latitude],longitude = filter_houses[:longitude],zip = filter_houses[:zip])

Давайте построим каждый кластер как отдельный цвет.

In [ ]:
clusters_figure = plot(legend = false)
for i = 1:k
    clustered_houses = df[df[:cluster].== i,:]
    xvals = clustered_houses[:latitude]
    yvals = clustered_houses[:longitude]
    scatter!(clusters_figure,xvals,yvals,markersize=4)
end
xlabel!("Latitude")
ylabel!("Longitude")
title!("Houses color-coded by cluster")
display(clusters_figure)

А теперь давайте попробуем раскрасить их по почтовому индексу.

In [ ]:
unique_zips = unique(filter_houses[:zip])
zips_figure = plot(legend = false)
for uzip in unique_zips
    subs = filter_houses[filter_houses[:zip].==uzip,:]
    x = subs[:latitude]
    y = subs[:longitude]
    scatter!(zips_figure,x,y)
end
xlabel!("Latitude")
ylabel!("Longitude")
title!("Houses color-coded by zip code")
display(zips_figure)

Давайте посмотрим на два сюжета рядом.

In [ ]:
plot(clusters_figure,zips_figure,layout=(2, 1))

Не совсем! но почти... Теперь мы знаем, что почтовые индексы назначаются не случайно!

### Пример 2: Ближайший сосед с KDTree

Для этого примера давайте начнем с загрузки пакета `NearestNeighbors`.

In [ ]:
]add NearestNeighbors

In [ ]:
using NearestNeighbors

С этим пакетом мы будем искать `knearest` одного из домов.

In [ ]:
knearest = 10
id = 70 # try changing this
point = X[:,id]

Теперь мы можем построить `KDTree` и использовать` knn` для поиска ближайших соседей `point`!

In [ ]:
kdtree = KDTree(X)
idxs, dists = knn(kdtree, point, knearest, true)

Сначала мы создадим участок со всеми домами одного цвета,

In [ ]:
x = filter_houses[:latitude];
y = filter_houses[:longitude];
scatter(x,y);

а затем наложить данные, соответствующие ближайшим соседям точки, другим цветом.

In [ ]:
x = filter_houses[idxs,:latitude];
y = filter_houses[idxs,:longitude];
scatter!(x,y)

Есть те ближайшие соседи в красном!

Мы можем увидеть районы для соседних домов, используя индексы `idxs` и функцию `:city` для индексации в `DataFrame`` filter_houses`.

In [ ]:
cities = filter_houses[idxs,:city]

### Пример 3: PCA для уменьшения размерности

Попробуем уменьшить размеры данных о цене / площади из набора данных домов.

Мы можем начать с захвата квадратных метров и цен на дома и хранения их в «массиве».

In [ ]:
F = filter_houses[[:sq__ft,:price]]
F = convert(Array{Float64,2},F)'

Вспомните, как выглядят данные, когда мы строим цены на жилье на основе квадратных метров.

In [ ]:
scatter(F[1,:],F[2,:])
xlabel!("Square footage")
ylabel!("Housing prices")

Мы можем использовать пакет MultivariateStats для запуска PCA

In [ ]:
]add MultivariateStats

In [ ]:
using MultivariateStats

Используйте `fit`, чтобы соответствовать модели

In [ ]:
M = fit(PCA, F)

Обратите внимание, что вы можете выбрать максимальное измерение нового пространства, установив `maxoutdim`, и вы можете изменить метод, например, на`: svd` со следующим синтаксисом.

```julia
fit(PCA, F; maxoutdim = 1,method=:svd)
```

Кажется, что мы получаем только одно измерение с PCA! Давайте используем `transform` для отображения всех наших 2D-данных в` F` в `1D` с нашей моделью` M`.

In [ ]:
y = transform(M, F)

Давайте использовать `reconstruct`, чтобы поместить наши теперь 1D данные,` y`, в форму, которую мы можем легко наложить (`Xr`) с нашими 2D данными в` F` вдоль основного направления / компонента.

In [ ]:
Xr = reconstruct(M, y)

И теперь мы создаем это наложение, где мы можем видеть точки вдоль основного компонента красным.

(Каждая синяя точка однозначно отображается на какую-то красную точку!)

In [ ]:
scatter(F[1,:],F[2,:])
scatter!(Xr[1,:],Xr[2,:])

### Пример 4: Узнайте, как построить простой многослойный персептрон на наборе данных MNIST

MNIST от: https://github.com/FluxML/model-zoo/blob/master/mnist/mlp.jl

Давайте начнем с загрузки `Flux`, явного импорта нескольких вещей из` Flux` и переноса функции `repeat` в нашу область видимости.

In [ ]:
]add Flux

In [ ]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated

Теперь мы можем сохранить все изображения MNIST в «imgs» и взять пик в этом векторе, чтобы увидеть, как выглядят данные

In [ ]:
imgs = MNIST.images()
imgs[3]

Давайте посмотрим на тип отдельного изображения.

In [ ]:
typeof(imgs[3])

#### Реорганизация нашего массива изображений

Мы видим, что это двумерный массив, который хранит ColorTypes. Чтобы было легче работать с этими данными, давайте конвертируем все `ColorTypes` в числа с плавающей запятой.

In [ ]:
fpt_imgs = float.(imgs)

Теперь мы можем видеть, что `imgs [3]` выглядит как массив чисел с плавающей точкой, а не как массив цветов!

In [ ]:
fpt_imgs[3]

**Давайте сложим изображения, чтобы создать один большой двумерный массив, `X`, который хранит данные для каждого изображения в виде столбца.**

Чтобы сделать это, мы можем сначала использовать `reshape`, чтобы распутать каждое изображение, создавая одномерный массив (` Вектор`) с плавающей точкой из 2D массива (`Matrix`) с плавающей точкой.

In [ ]:
unraveled_fpt_imgs = reshape.(fpt_imgs, :);
typeof(unraveled_fpt_imgs)

(Обратите внимание, что `Vector` является псевдонимом для 1D` Array`.)

In [ ]:
Vector

Это делает `unraveled_fpt_imgs` вектором `Vector`ов где `imgs [3]` сейчас

In [ ]:
unraveled_fpt_imgs[3]

После использования `reshape` для получения` Vector` из `Vector`ов, мы можем использовать` hcat` для построения `Matrix`,` X`, из `unraveled_fpt_imgs`, где` Vector`s, сохраненный в `unraveled_fpt_imgs`, станут столбцами `X`.

Обратите внимание, что мы используем команду «splat» ниже, `...`, которая позволяет вам передавать все элементы объекта в функцию, вместо того, чтобы просто передать сам объект.

In [ ]:
X = hcat(unraveled_fpt_imgs...)

#### Как вернуться к изображениям из этого 2D-массива

Теперь каждый столбец в X - это изображение, преобразованное в вектор с плавающей точкой. Давайте выберем один столбец и посмотрим, что это за цифра.

Давайте попробуем просмотреть второе изображение в исходном массиве `imgs`, взяв второй столбец` X`

In [ ]:
onefigure = X[:,2]

Мы «изменим» этот массив в 2D, массив 28x28,

In [ ]:
t1 = reshape(onefigure,28,28)

и, наконец, используйте `colorview` из пакета` Images` для просмотра рукописной цифры.

In [ ]:
using Images

In [ ]:
colorview(Gray, t1)

*Наши данные в рабочем состоянии!*

Чтобы наша машина выучила цифру, с которой связано каждое изображение, нам нужно будет обучить ее, используя правильные ответы. Поэтому мы будем использовать «метки», связанные с этими изображениями из MNIST.

In [ ]:
labels = MNIST.labels() # the true labels

One-hot-encode the labels with `onehotbatch`

In [ ]:
Y = onehotbatch(labels, 0:9)

который дает двоичный вектор индикатора для каждой фигуры

Задаем сеть

In [ ]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

Определить функции потерь и точность

In [ ]:
loss(x, y) = Flux.crossentropy(m(x), y)
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

Используйте `X` для создания наших обучающих данных, а затем объявите нашу функцию оценки:

In [ ]:
dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM(Flux.params(m))

До сих пор мы определили наши данные обучения и наши функции оценки.

Давайте посмотрим на сигнатуру функции Flux.train!

In [ ]:
?Flux.train!

**Теперь мы можем обучить нашу модель и посмотреть на точность в дальнейшем.**

In [ ]:
Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

Теперь, когда мы обучили нашу модель, давайте создадим тестовые данные, `tX`,

In [ ]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...)

и запустим нашу модель на одном из изображений из `tX`

In [ ]:
test_image = m(tX[:,1])

In [ ]:
argmax(test_image) - 1

Самый большой элемент `test_image` - это восьмой элемент, поэтому наша модель говорит, что test_image - это" 7 ".

Теперь мы можем посмотреть на исходное изображение.

In [ ]:
# using Images
t1 = reshape(tX[:,1],28,28)
colorview(Gray, t1)

Крута!

### Пример 5: Линейная регрессия в julia (мы напишем наш собственный код Юлии и код Python)

Попробуем найти лучшее соответствие строки из следующих данных:

In [ ]:
xvals = repeat(1:0.5:10,inner=2)
yvals = 3 .+ xvals + 2*rand(length(xvals)) .- 1
scatter(xvals,yvals,color=:black,leg=false)

Мы хотим провести линию через эти данные.

Давайте напишем функцию Julia, чтобы сделать это.

In [ ]:
function find_best_fit(xvals,yvals)
    meanx = mean(xvals)
    meany = mean(yvals)
    stdx = std(xvals)
    stdy = std(yvals)
    r = cor(xvals,yvals)
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
end

Чтобы соответствовать линии, нам просто нужно найти наклон и точку пересечения y (a и b).

Затем добавьте это соответствие к существующему сюжету!

In [ ]:
a,b = find_best_fit(xvals,yvals)
ynew = a * xvals .+ b

In [ ]:
plot!(xvals,ynew)

Давайте сгенерируем намного больший набор данных,

In [ ]:
xvals = 1:100000;
xvals = repeat(xvals,inner=3);
yvals = 3 .+ xvals + 2*rand(length(xvals)) .- 1;

In [ ]:
@show size(xvals)
@show size(yvals)

и теперь мы можем определить, сколько времени потребуется, чтобы найти соответствие этим данным.

In [ ]:
@time a,b = find_best_fit(xvals,yvals)

Теперь мы напишем тот же код, используя Python

In [ ]:
# Они установлены, раз Вы работаете в Jupyter
using PyCall
using Conda

In [ ]:
py"""
import numpy
def find_best_fit_python(xvals,yvals):
    meanx = numpy.mean(xvals)
    meany = numpy.mean(yvals)
    stdx = numpy.std(xvals)
    stdy = numpy.std(yvals)
    r = numpy.corrcoef(xvals,yvals)[0][1]
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
"""

In [ ]:
find_best_fit_python = py"find_best_fit_python"

In [ ]:
xpy = PyObject(xvals)
ypy = PyObject(yvals)
@time a,b = find_best_fit_python(xpy,ypy)

**Давайте использовать пакет сравнительного анализа, чтобы рассчитать эти два.**

In [ ]:
using BenchmarkTools

In [ ]:
@btime a,b = find_best_fit_python(xvals,yvals)

In [ ]:
@btime a,b = find_best_fit(xvals,yvals)

Здесь смысл не в том, что "фу, какой медленный питон", ведь с нужными инструментами его можно нехило разогнать. Тут фишка скорее в том, что с Джулией Вы можете реализовывать идеи на скорую руку, получая при этом высокую производительность, а для того, чтоб программирование в общем и datascience в частности были ближе к исскуству, очень важно, чтобы долгие вычисления не спугнули вдохновение!